In [1]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy.io import fits


In [2]:
# This cell contains code taken from my research made by Professor Sean Johnson that reads in the files from the dataset

# Read int the quasar table (actually much shorter quick-look version) 
quasars = Table.read('GO-12289 target table - all.csv', format='ascii')
print(quasars['name', 'redshift', 'COS_filename1'])

# Function that reads in the COS spectrum and returns a version rebinned to a 1 Angstrom pixel.
def readCOS(filename):

    # Read in the COS spectrum
    spec_direct = Table.read('GO-12289_spectra/{}'.format(filename))

    # Reformat as an astropy Table for convenience
    spec = Table()
    wave = spec_direct['WAVELENGTH']
    spec['wave'] = np.array(spec_direct['WAVELENGTH'])[0, :]
    spec['flux'] = np.array(spec_direct['FLUX'])[0, :]
    spec['error'] = np.array(spec_direct['ERROR'])[0, :]
    spec['nPix'] = np.zeros(len(spec))
    spec = spec[(spec['wave'] > 1100) & (spec['wave'] < 1900)]
    
  
    # Now rebin the spectra to 1 Angstrom intervals: combining 
    dwave = 1
    minwave = np.min(spec['wave'])
    maxwave = np.max(spec['wave'])

    # Create the wavelength array for the rebinned spectrum
    waveArray = np.arange(minwave, maxwave, dwave)
    spec_rebinned = Table()
    spec_rebinned['wave'] = waveArray
    # Fillin dummy values for flux, error, and number of pixels for now.
    spec_rebinned['flux'] = np.zeros(len(waveArray))
    spec_rebinned['error'] = np.zeros(len(waveArray))
    spec_rebinned['nPix'] = np.zeros(len(waveArray))

    # Look through the rebinned spectrum and find the flux, error, and nPix at each wavelength
    for pixel in spec_rebinned:

        # Identify pixels in the original spectrum that should contribute to this pixel in the rebinned spectrum
        index = np.where((spec['wave'] >= pixel['wave'] - dwave/2) & (spec['wave'] < pixel['wave'] + dwave/2))

        # Calculate the mean flux
        pixel['flux'] = np.nanmean(spec[index]['flux'])

        # Calculate the standard deviation which we can use for the error
        pixel['error'] = np.nanstd(spec[index]['error'])

        # Just in case we need it, calculate the number of original pixels contributing to this pixel.
        pixel['nPix'] = len(index[0])

    # Throw out data between 1150 and 1310 Angstroms because it is impacted by Geocoronal emission.
    spec_rebinned = spec_rebinned[(spec_rebinned['wave'] < 1150) | (spec_rebinned['wave'] > 1320)]
    return spec_rebinned


FileNotFoundError: [Errno 2] No such file or directory: 'GO-12289 target table - all.csv'